In [28]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from scipy import stats as st
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from pylab import mpl
import datetime as dt

tickers = ['AAPL','FB', 'C', 'DIS']

data = pdr.get_data_yahoo(tickers, start="2010-01-01", end=dt.date.today())['Close']

#returns = data.pct_change().dropna()


log_re = np.log(data/data.shift(1))

log_re.sort_index(ascending=False, inplace = True)

log_re

Symbols,AAPL,FB,C,DIS
Date,,,,
2021-12-17,-0.006523,-0.003320,-0.025372,0.000067
2021-12-16,-0.040055,-0.019984,0.013684,-0.011031
2021-12-15,0.028110,0.023454,-0.006618,0.008681
2021-12-14,-0.008056,-0.002245,0.007615,-0.008881
2021-12-13,-0.020891,0.014272,-0.008768,-0.015043
...,...,...,...,...
2010-01-08,0.006626,NaN,-0.016575,0.001570
2010-01-07,-0.001850,NaN,0.002743,0.000314
2010-01-06,-0.016034,NaN,0.030686,-0.005328


In [20]:
log_re = log_re.iloc[0::30,:]
log_re


Symbols,AAPL,FB,C,DIS
Date,,,,
2021-12-17,-0.006523,-0.003320,-0.025372,0.000067
2021-11-04,-0.003505,0.012675,-0.029414,0.001175
2021-09-23,0.006697,0.007981,0.037957,0.014862
2021-08-11,0.001784,-0.003245,0.017926,0.005744
2021-06-29,0.011435,-0.010600,-0.025924,-0.015065
...,...,...,...,...
2010-07-13,-0.021569,NaN,0.045192,0.019936
2010-05-28,0.013837,NaN,-0.015038,-0.028030
2010-04-16,-0.006125,NaN,-0.053374,-0.015783


In [21]:
log_re = log_re.iloc[0:100,:]

avg_return = log_re.mean(axis = 1)

avg_return = pd.DataFrame(avg_return, columns = ['Return'])
#avg_return.set_axis(['Return'], axis=1, inplace=True)

avg_return['Date Order'] = range(1,101,1)
avg_return.sort_values(by = 'Return', inplace = True)
avg_return

,Return,Date Order
Date,,
2020-03-09,-0.106116,16
2011-10-31,-0.038685,86
2020-04-21,-0.037009,15
2011-05-11,-0.030576,90
2018-11-12,-0.027606,27
...,...,...
2014-02-07,0.017961,67
2015-01-22,0.021360,59
2013-04-10,0.024741,74


In [22]:
data10 = avg_return.iloc[:10,:]

d = 0.96; k = 100

w=pd.DataFrame([(0.96**(i-1))*(1-d)/(1-d**k) for i in data10['Date Order'].values], columns = ['Weight'])

w.index = data10.index

acc_w = w['Weight'].cumsum()
acc_w = pd.DataFrame(acc_w)
acc_w.index = data10.index
acc_w.set_axis(['Sum Weight'], axis=1, inplace=True)

acc_w

,Sum Weight
Date,
2020-03-09,0.022056
2011-10-31,0.023322
2020-04-21,0.046296
2011-05-11,0.047372
2018-11-12,0.061448
2016-06-27,0.067670
2010-04-16,0.068415
2014-07-31,0.071653
2018-02-27,0.082672


In [23]:
table=pd.concat([data10,w,acc_w],axis=1)
table

,Return,Date Order,Weight,Sum Weight
Date,,,,
2020-03-09,-0.106116,16,0.022056,0.022056
2011-10-31,-0.038685,86,0.001266,0.023322
2020-04-21,-0.037009,15,0.022975,0.046296
2011-05-11,-0.030576,90,0.001075,0.047372
2018-11-12,-0.027606,27,0.014077,0.061448
2016-06-27,-0.025780,47,0.006222,0.067670
2010-04-16,-0.025094,99,0.000745,0.068415
2014-07-31,-0.022823,63,0.003238,0.071653
2018-02-27,-0.021177,33,0.011019,0.082672


In [24]:
acc = np.array([0.039831, 0.061004])
r=np.array([-0.021248, -0.020450])
accnew = np.array([0.039831, 0.05, 0.061004])
f = interp1d(x= acc, y = r, kind = 'linear')
Var = f(0.05)

print('Weighted VAR at 5% is {:.4f}'.format(Var))



Weighted VAR at 5% is -0.0209


In [25]:
d = 0.96; k = 100

w1=pd.DataFrame([(0.96**(i-1))*(1-d)/(1-d**k) for i in avg_return['Date Order'].values], columns = ['Weight'])

w1.index = avg_return.index

acc_w1 = w1['Weight'].cumsum()
acc_w1 = pd.DataFrame(acc_w1)
acc_w1.index = avg_return.index
acc_w1.set_axis(['Sum Weight'], axis=1, inplace=True)

table2 = pd.concat([avg_return , w1 ,acc_w1],axis=1)

table2



,Return,Date Order,Weight,Sum Weight
Date,,,,
2020-03-09,-0.106116,16,0.022056,0.022056
2011-10-31,-0.038685,86,0.001266,0.023322
2020-04-21,-0.037009,15,0.022975,0.046296
2011-05-11,-0.030576,90,0.001075,0.047372
2018-11-12,-0.027606,27,0.014077,0.061448
...,...,...,...,...
2014-02-07,0.017961,67,0.002750,0.987893
2015-01-22,0.021360,59,0.003812,0.991705
2013-04-10,0.024741,74,0.002067,0.993772


In [27]:

c=0.90
for i in range(k-1):
	if table2.iloc[i,3]<1-c<=table2.iloc[i+1,3]:
		acci,accj=table2.iloc[i,3],table2.iloc[i+1,3]
		ri,rj=table2.iloc[i,0],table2.iloc[i+1,0]
	elif 1-c<table2.iloc[0,3]:
		print('数据不足！')
		break	
acc=np.array([acci,accj])
r=np.array([ri,rj])
accnew=np.array([acci,1-c,accj])
f=interp1d(x=acc,y=r,kind='linear')
print('{}%的VaR为{:.4f}'.format(100*c,abs(f(accnew)[1])))


90.0%的VaR为0.0145
